# SWMM database
Create and export hydraulic head simulations for synthetic or real rainfalls

## Imports and directories

Imports

In [2]:
#Libraries
import numpy as np
import plotly.graph_objects as go
import pickle
import subprocess
import os

import swmmtoolbox.swmmtoolbox as swm

#Custom utils
import utils_data as ud

Directories

In [3]:
datfile_home = r'C:\Users\agarzondiaz\surfdrive\Year 2\Paper 2 - 3.0\data\rainfall_dats'
simulations_path = r'C:\Users\agarzondiaz\surfdrive\Year 2\Paper 2 - 3.0\data\simulations'
inp_path = simulations_path + '\\Tuindorp_Right_Part.inp'

## Synthetic rainfalls

1. Create synthetic rainfalls

Alternating blocks method

In [2]:
rainfalls = []
N_rainfalls = 200
for i in range(N_rainfalls):
  rand_A = np.random.randint(10000, 35000)
  rand_B = np.random.uniform()*(20) + 30
  rand_dur = np.random.randint(4,36)*5


  random_idf ={ 'A':rand_A,
                'B':rand_B,
                'n':1.09}

  specs_rain = ['Synt_'+str(i), random_idf, rand_dur]
  one_random_rain = ud.altblocks(random_idf, dur = rand_dur, dt=5)
  rainfalls.append((specs_rain, one_random_rain))


Block rainfalls

In [3]:
block_rainfalls = []
# specs_rain = ['Block_'+str(i)]
values =    [ 0, 20,  0, 30,  0, 40,  0, 30,  0, 20,  0, 10,  0]
durations = [30, 30, 30, 30, 40, 30, 40, 30, 30, 30, 30, 30, 50]
block_rain = ud.rain_blocks(values, durations, dt=5)
# block_rainfalls.append((specs_rain, one_random_rain))

2. Created rainfalls visualization

In [4]:
# Visualisation of synthetic hietographs
fig = go.Figure()
for i in range(3):
    rain = block_rain#rainfalls[i][1]
    # rain = one_random_rain
    fig.add_trace(ud.hietograph(rain))
fig.show()

In [5]:
string_rain = ["".join(i) for i in ud.new_rain_lines_dat(block_rain, name_new_rain = 'R1') ]
with open('.\\rainfall_dats\\block.dat', 'w') as f:
    f.writelines(string_rain)

In [11]:
for idx in range(20):
    string_rain = ["".join(i) for i in ud.new_rain_lines_dat(rainfalls[idx][1], name_new_rain = 'R1') ]
    with open('.\\rainfall_dats\\synth_'+ str(idx) +'.dat', 'w') as f:
        f.writelines(string_rain)

## Real rainfalls

In [12]:
# %% Read rainfall in each pixel
with open('.\\real_rainfalls\\events_pixel1_2014_5h.pk', 'rb') as f:
  pixel1 = pickle.load(f)
with open('.\\real_rainfalls\\events_pixel2_2014_5h.pk', 'rb') as f:
  pixel2 = pickle.load(f)
with open('.\\real_rainfalls\\events_pixel3_2014_5h.pk', 'rb') as f:
  pixel3 = pickle.load(f)

In [13]:
fig = go.Figure()

rain = pixel2[40]
fig.add_trace(ud.hietograph(rain))
fig.show()

In [14]:
#Temporarily considering only pixel2
N_rains = 20
for i in range(N_rains):
    if np.array(list((pixel2[i].values()))).max() > 10:
        string_rain = ["".join(line) for line in ud.get_new_rain_lines_real(pixel2[i], name_new_rain = 'R1') ]
        with open('.\\rainfall_dats\\real_'+ str(i) +'.dat', 'w') as f:
            f.writelines(string_rain)

## Run SWMM 

In [21]:
list_of_rain_datfiles = os.listdir(datfile_home)


for event in list_of_rain_datfiles:
    rain_event_path = datfile_home + '\\' + event
    
    inp =  ud.get_lines_from_textfile(inp_path)
    dat =  ud.get_lines_from_textfile(rain_event_path)
    

    for ln, line in enumerate(inp):
        splitted_line_dat = dat[0].split('\t')
        new_date = ''.join([splitted_line_dat[2], '/',splitted_line_dat[3],'/', splitted_line_dat[1]])
        splitted_line_dat_last = dat[-1].split('\t')
        new_last_date = ''.join([splitted_line_dat_last[2], '/',splitted_line_dat_last[3],'/', splitted_line_dat_last[1]])

        new_last_time = ''.join([splitted_line_dat_last[4], ':',splitted_line_dat_last[5], ':', '00'])
        if 'START_DATE' in line:
            inp[ln] = line.replace(line.split()[-1], new_date)
            
        elif 'END_DATE' in line:
            inp[ln] = line.replace(line.split()[-1], new_last_date)
            
        elif 'END_TIME' in line:
            inp[ln] = line.replace(line.split()[-1], new_last_time)
            

        elif 'PLACEHOLDER1' in line:
            inp[ln] = line.replace('PLACEHOLDER1', '\\'.join((datfile_home, event)))
            # print(inp[ln])

            
        # elif 'PLACEHOLDER2' in line:
        #     inp[ln] = line.replace('PLACEHOLDER2', '\\'.join((datfile_home, event, 'datfiles',
        #                                                     'rain' +str(k[1]) + '.dat')))
        # elif 'PLACEHOLDER3' in line:
        #     inp[ln] = line.replace('PLACEHOLDER3', '\\'.join((datfile_home, event, 'datfiles',
        #                                                     'rain' +str(k[2]) + '.dat')))

    nf = '\\'.join((simulations_path, event.replace('.dat', '')))
    os.mkdir(nf)

    with open(nf + '\\model.inp', 'w') as fh:
        for line in inp:
            fh.write("%s" % line)   


    subprocess.run([r'C:\Program Files (x86)\EPA SWMM 5.1.015\swmm5.exe',
                    nf+'\\model.inp',
                    nf+'\\model.rpt',
                    nf+'\\model.out'])

    with open(nf + '\\'+ event, 'w') as fh:
        for line in dat:
            fh.write("%s" % line)

        

## Extract and export hydraulic head

In [3]:
inp =  ud.get_lines_from_textfile(inp_path)

working_out = simulations_path + '\\block' +  '\\Model.out'

out_timeseries  = swm.extract(working_out, 'node,,Hydraulic_head')

Export information to a pickle file

In [12]:
#Pickle hydraulic head 
with open(r'C:\Users\agarzondiaz\surfdrive\Year 2\Paper 2 - 3.0\data\simulations\block\rainblock_head_TuindorpRP.pk', 'wb') as handle:
    pickle.dump(out_timeseries, handle, protocol=pickle.HIGHEST_PROTOCOL)

### Extra code

In [9]:
# line_where = {header.replace('\n',''):number for number, header in enumerate(inp) if '[' in header}

# Node features 
# headers_of_interest = ['[OUTFALLS]', '[JUNCTIONS]']
# o_names, o_elevation = ud.extract_info_inp(inp, line_where,headers_of_interest[0])
# o_j_names, o_j_elevation = ud.extract_info_inp(inp, line_where,headers_of_interest[1],  o_names, o_elevation)
# node_names, node_elevation = ud.extract_info_inp(inp, line_where, headers_of_interest[2], o_j_names, o_j_elevation)